### Naive Bayes Classifier sentiment analysis with scikit-learn

In this notebook, a Naive Bayes Classifier is used to classify french and german tweets. In the provided data, a sentiment analysis has already been done on the french and german tweets, but only using the smileys. Now, they classified tweets are used to classify the other tweets that do not have these smileys and that have been classified as neutral. One problem is that we do not have tweets that have been classified as neural BECAUSE they are neutral. Indeed, most neutral tweets are in fact unclassified tweets. Because of class imbalances, we only consider subsets of positive tweets.

Import the libraries

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize.casual import TweetTokenizer
from nltk.stem.snowball import FrenchStemmer, GermanStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score
from sklearn import preprocessing

Use the snowball algorithm (http://snowballstem.org/) to stem french and german words. A casual tokenizer is used as it is more adapted to Twitter data. Stopwords are also loaded for both languages. A simple regular expression is compiled to detect links.

In [2]:
fr_stemmer = FrenchStemmer()
de_stemmer = GermanStemmer()
tokenizer = TweetTokenizer(strip_handles=True)
stop = stopwords.words('french') + stopwords.words('german')
re_url = re.compile('^(http[s]?://)?[\S]+\.\S[\S]+[/?#][\S]+$')

Let's now define some useful function to make the code easier to read

Remove stopwords, URLs and single character words

In [3]:
def filter_word(word):
    return word not in stop and not re_url.match(word) and len(word) > 1

Stem th word after removing a possible hashsign

In [4]:
def clean_word(stemmer, word):
    return stemmer.stem(word.replace('#', ''))

Convert a list of lists of items to a list of items (2D -> 1D)

In [5]:
def flatten(lst):
    return [item for sublist in lst for item in sublist]

Transform a tweet into a list of cleaned, stemmed words

In [6]:
def process(x, stemmer):
    return ' '.join(clean_word(stemmer, word) for word in tokenizer.tokenize(x) if filter_word(word))

This function generate another function that will be used to find what features are present in a tweet.

In [7]:
#def generate_callback(features):
#    def extract_features(text):
#        extract = {}#[None]*len(features)
#        for feature in features:
#            extract[feature] = feature in text
#        return extract
    
#    return extract_features

Parse one month of data from a CSV file for a given language. Tweets are separated by sentiment in three different Pandas DataFrames.

In [8]:
def parse_month(curr_pos, curr_neg, curr_neu, lang, month):
    raw = pd.read_csv('processed/tweets_non-en_msg_{}.csv'.format(month), escapechar='\\')
    raw.columns=['source_location', 'lang', 'main', 'sentiment']

    filtered = raw[raw.lang == lang]

    pos = filtered[filtered.sentiment == 'POSITIVE']
    neg = filtered[filtered.sentiment == 'NEGATIVE']
    neu = filtered[filtered.sentiment == 'NEUTRAL']
    
    stemmer = fr_stemmer if lang == 'fr' else de_stemmer if lang == 'de' else None

    pos = pos[['source_location', 'sentiment']].assign(tokenized=pos['main'].apply(lambda x: process(x, stemmer)))
    neg = neg[['source_location', 'sentiment']].assign(tokenized=neg['main'].apply(lambda x: process(x, stemmer)))
    neu = neu[['source_location', 'sentiment']].assign(tokenized=neu['main'].apply(lambda x: process(x, stemmer)))
    
    if curr_pos is None:
        return (pos, neg, neu)
    else:
        return (pd.concat([curr_pos, pos], copy=False),
                pd.concat([curr_neg, neg], copy=False),
                pd.concat([curr_neu, neu], copy=False))

Train a Naive Bays classifier

In [9]:
def fit_classifier(X_train, y_train):
    count_vec = CountVectorizer()
    X_features = count_vec.fit_transform(X_train)
    
    tf_transformer = TfidfTransformer(use_idf=False)
    X_tf = tf_transformer.fit_transform(X_features)
    
    return (MultinomialNB().fit(X_tf, y_train), count_vec, tf_transformer)

Print the accuracy, recall and f1 score (based on accuracy and recall)

In [10]:
def score(classifier, count_vec, tf_transformer, X_test, y_test):
    # Binarize and test
    binary = preprocessing.LabelBinarizer()
    y_pred, features = predict(classifier, count_vec, tf_transformer, X_test)
    
    y_test = binary.fit_transform(y_test)
    y_pred = binary.fit_transform(y_pred.tolist())
    
    print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))
    print("Recall: {}".format(recall_score(y_test, y_pred)))
    print("F1 Score: {}".format(f1_score(y_test, y_pred)))

Predict using the classifier

In [11]:
def predict(classifier, count_vec, tf_transformer, X):
    X_new_features = count_vec.transform(X)
    X_new_tf = tf_transformer.transform(X_new_features)
    return classifier.predict(X_new_tf), X_new_features

Classify the neutral tweets using the positive and negative tweets that were classified using the smileys (:), :(). Note that tweets for which the features are all zeros will stay neutral.

In [12]:
def classify_neutral_tweets(classifier, count_vec, tf_transformer, neutral):
    # Classify tweets
    predicted, features = predict(classifier, count_vec, tf_transformer, neutral.tokenized.tolist())
    neutral = neutral.assign(sentiment=predicted)
    
    # Remove unclassifiable tweets (their features is the zero matrix)
    stay_neutral = np.array(features.sum(axis=1) == 0).squeeze()
    neutral['sentiment'][stay_neutral] = 'NEUTRAL'
    
    return neutral

Classify the tweets for the given months

In [13]:
def sentiments_by_month(dataset, lang, months):
    X = dataset.tokenized.tolist()
    y = dataset.sentiment.tolist()

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

    # Train
    classifier, count_vec, tf = fit_classifier(X_train, y_train)
    
    # Test
    score(classifier, count_vec, tf, X_test, y_test)

    output = {}
    
    for month in months:
        print("Classifying month {}...".format(month))
        positive, negative, neutral = parse_month(None, None, None, lang, month)
        output[month] = pd.concat([positive, negative, classify_neutral_tweets(classifier, count_vec, tf, neutral)])[['source_location', 'sentiment']]
        
    return output

Load the data from the CSVs. Store the tweets of different languages in different DataFrames. The <lang>_all DataFrames contains positive and negative tweets for a specific language. Empty tweets (after processing) are removed from these DataFrames.

In [14]:
fr_pos = None
fr_neg = None
fr_neu = None
de_pos = None
de_neg = None
de_neu = None

months = [
    'january',
    'february',
    'march',
    'april',
    'may',
    'june',
    'july',
    'august',
    'september',
    'october'
]

# fr
for month in months:
    fr_pos, fr_neg, fr_neu = parse_month(fr_pos, fr_neg, fr_neu, 'fr', month)

#fr_pos = fr_pos.sample(frac=1).reset_index(drop=True)   # Shuffle the DataFrame
fr_pos = fr_pos[0:len(fr_neg)]                          # Take only a subset due to class imbalance
fr_all = pd.concat([fr_pos, fr_neg], copy=False)
fr_all = fr_all[fr_all.tokenized != '']                 # Remove empty strings
fr_neu = fr_neu[fr_neu.tokenized != '']

# de
for month in months:
    de_pos, de_neg, de_neu = parse_month(de_pos, de_neg, de_neu, 'de', month)

#de_pos = de_pos.sample(frac=1).reset_index(drop=True)
de_pos = de_pos[0:len(de_neg)]
de_all = pd.concat([de_pos, de_neg], copy=False)
de_all = de_all[de_all.tokenized != '']
de_neu = de_neu[de_neu.tokenized != '']

In [15]:
fr_all.head()

,source_location,sentiment,tokenized
4,Delémont,POSITIVE,arrêt foutr merd 2016 svp merc beaucoup bon nu...
414,Baden,POSITIVE,nous bier pros neujahr ;)
1366,Lausanne,POSITIVE,on calcul le gen réveillent mainten alor bon a...
1439,Lausanne,POSITIVE,ça manqu peut-êtr peu poes ça mani d'optimis p...
1467,Lausanne,POSITIVE,bon anné coral te compliqu trop vi ;)


In [16]:
de_all.head()

,source_location,sentiment,tokenized
41,Zuerich,POSITIVE,so start neu jahr :-)
50,Biel/Bienne,POSITIVE,ja chonnt mach ;)
134,Solothurn,POSITIVE,solang stolp fall fuhrt gewiss rein ;-)
180,Biel/Bienne,POSITIVE,wir hoff ihr hattet gut start neu jahr ;-) wir...
216,Chur,POSITIVE,es gab schnee jahr :)


We want to store the results per month. Classify the tweets and keep only source_location and sentiment, per month.

In [17]:
fr_by_month = sentiments_by_month(fr_all, 'fr', months)

Accuracy: 0.7512165450121655
Recall: 0.6970443349753694
F1 Score: 0.734587929915639
Classifying month january...
Classifying month february...
Classifying month march...
Classifying month april...
Classifying month may...
Classifying month june...
Classifying month july...
Classifying month august...
Classifying month september...
Classifying month october...


In [27]:
columns = ['source_location', 'sentiment']
path = 'processed/sentiments/tweets_sentiments_bayes_fr_{}.csv'

for key, df in fr_by_month.items():
    f = open(path.format(key), 'w')
    f.write(pd.DataFrame(df.groupby(columns).size()).to_csv())
    f.close()

In [28]:
de_by_month = sentiments_by_month(de_all, 'de', months)

Accuracy: 0.7178683385579937
Recall: 0.7100213219616205
F1 Score: 0.7115384615384616
Classifying month january...
Classifying month february...
Classifying month march...
Classifying month april...
Classifying month may...
Classifying month june...
Classifying month july...
Classifying month august...
Classifying month september...
Classifying month october...


In [29]:
columns = ['source_location', 'sentiment']
path = 'processed/sentiments/tweets_sentiments_bayes_de_{}.csv'

for key, df in de_by_month.items():
    f = open(path.format(key), 'w')
    f.write(pd.DataFrame(df.groupby(columns).size()).to_csv())
    f.close()